In [ ]:
import pandas as pd
import networkx as nx
from BeyondBlooms2024.config import name_dict, color_dict
from BeyondBlooms2024.config_file import (ABUNDANCES_FILE, CCMN_CON_MAP_PATH, CON_LOUVAIN_META_PATH,CON_LOUVAIN_NETWORK_PATH, ENRICH,
NUM_PERMUTATIONS, NUM_SAMPLES, NUM_CORES, METADATA_FILE, PRUNED_PVAL_CCMN_PATH,PVAL_CCMN_PATH,ENRICHED_META_PATH, RANDOM_PVAL_CCMN_PATH)


In [ ]:
df_ss_egc_03L = pd.read_csv("CCMN_Paper_Submission/tables/woSeason_final_proj/EGC_03-L-Fig5_Project_Condtions__final__-12-|-01-|-02-_F4_EGCRaw_sstable.csv", sep=",").T
df_ss_egc_03L

In [ ]:
df_ss_f4_03L = pd.read_csv("CCMN_Paper_Submission/tables/woSeason_final_proj/F4_03-L-Fig5_Project_Condtions__final__-12-|-01-|-02-_F4Raw_sstable.csv", sep=",").T
df_ss_f4_03L

In [ ]:
df_ss_egc_10H = pd.read_csv("CCMN_Paper_Submission/tables/woSeason_final_proj/EGC_Test_10-H-Fig5_Project_Condtions__final__-06-|-07-|-08-_F4_EGCRaw_sstable.csv", sep=",").T
df_ss_egc_10H

In [ ]:
df_ss_f4_10H = pd.read_csv("CCMN_Paper_Submission/tables/woSeason_final_proj/F4_Test_10-H-Fig5_Project_Condtions__final__-06-|-07-|-08-_F4Raw_sstable.csv", sep=",").T
df_ss_f4_10H

In [ ]:
list_of_atlantic_3 = df_ss_f4_03L[(df_ss_f4_03L[0]==1) | (df_ss_f4_03L[1]==1) | (df_ss_f4_03L[2]==1) | (df_ss_f4_03L[3]==1)].index.to_list()[1:]
list_of_arctic_3 = df_ss_egc_03L[(df_ss_egc_03L[0]==1)].index.to_list()[1:]

In [ ]:
list_of_atlantic_10 = df_ss_f4_10H[(df_ss_f4_10H[0]==1) | (df_ss_f4_10H[1]==1) | (df_ss_f4_10H[2]==1)].index.to_list()[1:]
list_of_arctic_10 = df_ss_egc_10H[df_ss_egc_10H[0]==1].index.to_list()[1:]

In [ ]:
list_of_arctic_10

In [ ]:
df_ss_f4_03L_NON = pd.read_csv("CCMN_Paper_Submission/tables/woSeason_final/REla_with_env__mod_withoutSeason_03LW_-12-|-01-|-02-_tr0.01Raw_sstable.csv", sep=",").T
df_ss_f4_03L_NON

In [ ]:
list_of_atlantic_3_NON = df_ss_f4_03L_NON[(df_ss_f4_03L_NON[0]==1) | (df_ss_f4_03L_NON[1]==1) | (df_ss_f4_03L_NON[2]==1) | (df_ss_f4_03L_NON[3]==1)].index.to_list()[1:]

In [ ]:
df_ss_f4_10H_NON = pd.read_csv("CCMN_Paper_Submission/tables/woSeason_final/REla_with_env__mod_withoutSeason_10HS_-06-|-07-|-08-_tr0.02Raw_sstable.csv", sep=",").T
df_ss_f4_10H_NON

In [ ]:
list_of_atlantic_10_NON = df_ss_f4_10H_NON[(df_ss_f4_10H_NON[0]==1) | (df_ss_f4_10H_NON[1]==1) | (df_ss_f4_10H_NON[2]==1)].index.to_list()[1:]

In [ ]:
print("Origin Summer",len(list_of_atlantic_10_NON))
print("Atlantic Summer",len(list_of_atlantic_10))
print("Arctic Summer",len(list_of_arctic_10))
print("Origin Winter",len(list_of_atlantic_3_NON))
print("Atlantic Winter",len(list_of_atlantic_3))
print("Arctic Winter",len(list_of_arctic_3))

In [ ]:
import networkx as nx
import pandas as pd
# ccmn edges (nmi)
df_search_all = pd.read_csv(PVAL_CCMN_PATH , sep=";", index_col=0)
df_search_all

In [ ]:
df_search_meta = pd.read_csv(ENRICH, sep=",")
node_clu_dict_ = df_search_meta[["Nodes", "cluster_names"]]
node_clu_dict_.set_index("Nodes", inplace=True)
node_clu_dict =node_clu_dict_.to_dict()["cluster_names"]

In [ ]:
df_search_all["from_clu"]=df_search_all["from"].map(node_clu_dict)
df_search_all["to_clu"]=df_search_all["to"].map(node_clu_dict)

In [ ]:
def is_stable(x, l1):
    if x in l1:
        ret = 1
    else:
        ret = 0
    return ret

In [ ]:
clu_ = "03LW"
df_temp_clu = df_search_all[(df_search_all["from_clu"]== clu_) & (df_search_all["to_clu"]== clu_)]
df_temp_clu["1-corr"]= 1-df_temp_clu["corr"]
G = nx.from_pandas_edgelist(df_temp_clu, 'from', 'to', ['1-corr'], create_using=nx.DiGraph())
# Compute betweenness centrality for each node
betweenness = nx.betweenness_centrality(G, weight ="1-corr")

# Compute closeness centrality for each node
closeness = nx.closeness_centrality(G , distance ="1-corr")

# Display the betweenness centrality and closeness centrality for each node
list_of_nodes = []
list_of_BC = []
list_of_CC = []
for node in G.nodes():
    #print(f"Node {node}:")
    #print(f"Betweenness Centrality: {betweenness[node]}")
    #print(f"Closeness Centrality: {closeness[node]}")
    #print("------------------------")
    list_of_nodes.append(node)
    list_of_BC.append(betweenness[node])
    list_of_CC.append(closeness[node])
df_03 = pd.DataFrame(list_of_nodes)
df_03["Betweenness_Centrality"] = list_of_BC
df_03["Closeness Centrality"] = list_of_CC
df_03.rename(columns={0: "Asv"}, inplace=True)
df_03["atlantic"] = df_03["Asv"].apply(lambda x: is_stable(x, list_of_atlantic_3))
df_03["arctic"]= df_03["Asv"].apply(lambda x: is_stable(x, list_of_arctic_3))
df_03["origin"]= df_03["Asv"].apply(lambda x: is_stable(x, list_of_atlantic_3_NON))
df_03

In [ ]:
clu_2 = "10HS"
df_temp_clu2 = df_search_all[(df_search_all["from_clu"]== clu_2) & (df_search_all["to_clu"]== clu_2)]
df_temp_clu2["1-corr"]= 1-df_temp_clu2["corr"]
G2 = nx.from_pandas_edgelist(df_temp_clu2, 'from', 'to', ['1-corr'], create_using=nx.DiGraph())
# Compute betweenness centrality for each node
betweenness2 = nx.betweenness_centrality(G2, weight ="1-corr")

# Compute closeness centrality for each node
closeness2 = nx.closeness_centrality(G2 , distance ="1-corr")

# Display the betweenness centrality and closeness centrality for each node
list_of_nodes2 = []
list_of_BC2 = []
list_of_CC2 = []
for node in G2.nodes():
    #print(f"Node {node}:")
    #print(f"Betweenness Centrality: {betweenness[node]}")
    #print(f"Closeness Centrality: {closeness[node]}")
    #print("------------------------")
    list_of_nodes2.append(node)
    list_of_BC2.append(betweenness2[node])
    list_of_CC2.append(closeness2[node])
df_10 = pd.DataFrame(list_of_nodes2)
df_10["Betweenness_Centrality"] = list_of_BC2
df_10["Closeness Centrality"] = list_of_CC2
df_10.rename(columns={0: "Asv"}, inplace=True)
df_10["atlantic"] = df_10["Asv"].apply(lambda x: is_stable(x, list_of_atlantic_10))
df_10["arctic"]= df_10["Asv"].apply(lambda x: is_stable(x, list_of_arctic_10))
df_10["origin"]= df_10["Asv"].apply(lambda x: is_stable(x, list_of_atlantic_10_NON))
df_10

In [ ]:
df_10["cluster"] ="10HS"
df_03["cluster"] ="03LW"
df_10_3 = pd.concat([df_10, df_03])
df_10_3["Stable_State_Atlantic"] = df_10_3["atlantic"]
df_10_3["Stable_State_Arctic"] = df_10_3["arctic"]
df_10_3["Stable_State_Origin"] = df_10_3["origin"]
df_10_3.drop(columns=["atlantic"], inplace=True)
df_10_3.drop(columns=["arctic"], inplace=True)
df_10_3.drop(columns=["origin"], inplace=True)
df_10_3.set_index("Asv", inplace=True)

In [ ]:
atlantic_color= '#B82092'
arctic_color = '#2E7FD0'
origin_color = '#5D5FBE'

In [ ]:
df_10_3

In [ ]:
from matplotlib import pyplot as plt
SMALL_SIZE = 24
MEDIUM_SIZE = 24
BIGGER_SIZE = 24
MEDIUM= 24
TICK = 24
I_SIZE = 8
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=TICK)    # fontsize of the tick labels
plt.rc('ytick', labelsize=TICK)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)
plt.rcParams['axes.spines.left'] = True
plt.rcParams['axes.spines.right'] = True
plt.rcParams['axes.spines.top'] = True
plt.rcParams['axes.spines.bottom'] = True

In [ ]:
def get_non_ccm_asvs(df, li):
    nodes = set(df["to"].to_list()+df["from"].to_list())
    sli = set(li)
    set_d =sli.difference(nodes)
    sec = sli.intersection(nodes)
    return sec, set_d

In [ ]:
def create_table(list_of_asv=[], df_search=df_search_all, mod="Not_ALL"):
    if mod == "Not_ALL":
        list_of_ccm_asvs, list_non_ccm =get_non_ccm_asvs(df_search, list_of_asv)
        print("input asvs:", list_of_asv)
        print("Found in the CON ASVs:", len(list_of_ccm_asvs))
        print("Gone ASVs:", len(list_non_ccm))
        df_mew = pd.DataFrame(list(list_of_ccm_asvs))

    df_mew["cluster"] = df_mew.apply(lambda x:df_search[df_search["from"]==x[0]]["from_clu"].mode(), axis=1)
    df_mew["cluster"] = df_mew["cluster"]
    df_mew["number_connec_in"] = df_mew.apply(lambda x:df_search[(df_search["to"]==x[0]) & (df_search["from_clu"]==df_search["to_clu"])]["from"].count(), axis=1)
    df_mew["number_connec_out"] = df_mew.apply(lambda x:df_search[(df_search["from"]==x[0]) & (df_search["from_clu"]!=df_search["to_clu"])]["from"].count(), axis=1) #todo
    all_clusters = set(df_search["from_clu"].to_list()).union(set(df_search["to_clu"].to_list()))
    for clu in list(all_clusters):
        df_mew[f"from_clu_{clu}"] = df_mew.apply(lambda x:df_search[(df_search["from"]==x[0]) & (df_search["to_clu"]==clu)]["corr"].mean(), axis=1)
        df_mew[f"to_clu_{clu}"] = df_mew.apply(lambda x:df_search[(df_search["to"]==x[0]) & (df_search["from_clu"]==clu)]["corr"].mean(), axis=1)
        df_mew[f"number_clu_connec_{clu}"]= df_mew.apply(lambda x:df_search[(df_search["from"]==x[0]) & (df_search["to_clu"]==clu)]["from"].count(), axis=1)
    return df_mew

In [ ]:
df_list_of_atlantic_10_NON = create_table(list_of_asv=list_of_atlantic_10_NON)
df_list_of_atlantic_10_NON

In [ ]:
print("Origin Summer",len(list_of_atlantic_10_NON))
print("Atlantic Summer",len(list_of_atlantic_10))
print("Arctic Summer",len(list_of_arctic_10))
print("Origin Winter",len(list_of_atlantic_3_NON))
print("Atlantic Winter",len(list_of_atlantic_3))
print("Arctic Winter",len(list_of_arctic_3))

df_arctic_3 = create_table(list_of_asv=list_of_arctic_3)
sum_df_arctic_3 = df_arctic_3.describe()

df_atlantic_3 = create_table(list_of_asv=list_of_atlantic_3)
sum_df_atlantic_3 = df_atlantic_3.describe()

df_origin_3 = create_table(list_of_asv=list_of_atlantic_3_NON)
sum_df_origin_3 = df_origin_3.describe()

df_arctic_10 = create_table(list_of_asv=list_of_arctic_10)
sum_df_arctic_10 = df_arctic_10.describe()

df_atlantic_10 = create_table(list_of_asv=list_of_atlantic_10)
sum_df_atlantic_10 = df_atlantic_10.describe()

df_origin_10 = create_table(list_of_asv=list_of_atlantic_10_NON)
sum_df_origin_10 = df_origin_10.describe()
#####
list_of_all_asv_3 = df_ss_f4_03L_NON.index.to_list()
df_allASV_3 = create_table(list_of_asv=list_of_all_asv_3)
sum_df_allASV_3 = df_allASV_3.describe()

list_of_all_asv_10 = df_ss_f4_10H_NON.index.to_list()
df_allASV_10 = create_table(list_of_asv=list_of_all_asv_10)
sum_df_allASV_10 = df_allASV_10.describe()

custom_palette = {"arctic": arctic_color, "atlantic": atlantic_color, "origin": origin_color, "all ASVs": "darkgrey"}

In [ ]:
# Create a figure and subplots
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

df_arctic_3["env"]="arctic"
df_arctic_3["to_nmi"]=df_arctic_3['to_clu_03LW']
df_arctic_3["from_nmi"]=df_arctic_3['from_clu_03LW']

df_atlantic_3["env"]="atlantic"
df_atlantic_3["from_nmi"]=df_atlantic_3['from_clu_03LW']
df_atlantic_3["to_nmi"]=df_atlantic_3['to_clu_03LW']

df_origin_3["env"]="origin"
df_origin_3["from_nmi"]=df_origin_3['from_clu_03LW']
df_origin_3["to_nmi"]=df_origin_3['to_clu_03LW']

df_allASV_3["env"]="all ASVs"
df_allASV_3["from_nmi"]=df_allASV_3['from_clu_03LW']
df_allASV_3["to_nmi"]=df_allASV_3['to_clu_03LW']

df_arctic_10["env"]="arctic"
df_arctic_10["to_nmi"]=df_arctic_10['to_clu_10HS']
df_arctic_10["from_nmi"]=df_arctic_10['from_clu_10HS']

df_atlantic_10["env"]="atlantic"
df_atlantic_10["from_nmi"]=df_atlantic_10['from_clu_10HS']
df_atlantic_10["to_nmi"]=df_atlantic_10['to_clu_10HS']

df_origin_10["env"]="origin"
df_origin_10["from_nmi"]=df_origin_10['from_clu_10HS']
df_origin_10["to_nmi"]=df_origin_10['to_clu_10HS']

df_allASV_10["env"]="all ASVs"
df_allASV_10["from_nmi"]=df_allASV_10['from_clu_10HS']
df_allASV_10["to_nmi"]=df_allASV_10['to_clu_10HS']

df_l5 = pd.concat([df_arctic_3, df_atlantic_3, df_arctic_10, df_atlantic_10,df_origin_3,df_origin_10,df_allASV_3, df_allASV_10])
print(df_l5.head())
# Define the custom sorting order
custom_order = ["all ASVs", "origin", "atlantic", "arctic"]
# Convert the 'color' column to a categorical data type with custom ordering
df_l5['env'] = pd.Categorical(df_l5['env'], categories=custom_order, ordered=True)
df_l5.rename(columns={'to_nmi':'edge weight nmi (out)'}, inplace=True)
# Sort the DataFrame based on the 'color' column
df_sorted = df_l5.sort_values(by='env')

closedict = df_10_3["Closeness Centrality"].to_dict()
betdict = df_10_3["Betweenness_Centrality"].to_dict()
df_l5["Closeness Centrality"] =df_l5[0].map(closedict)
df_l5["Betweenness_Centrality"] =df_l5[0].map(betdict)
# List of columns for plotting
columns = ["Closeness Centrality","Betweenness_Centrality",'number_connec_in', 'number_connec_out', 'from_nmi', 'to_nmi']
columns = ["Closeness Centrality", 'edge weight nmi (out)']
quartile_colors = ['lightblue', 'royalblue', 'lightblue']
# Create violin plots for each column

In [ ]:
df_l5

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(20, 12))
df_l5_reset = df_l5.reset_index(drop=True)
for i, column in enumerate(columns):
    ax = sns.violinplot(data=df_l5_reset, x= "cluster",  y=column, ax=axes[i], hue="env", palette=custom_palette )
    #ax.gca().legend.remove()
    #axes[i].set_title(column)
    #axes[i].set_xlabel("")  # Remove x-label for better visualization
        # Get the handles and labels for the first subplot to create the legend
    if i == 0:
        handles, _ = axes[i].get_legend_handles_labels()

# Create a custom legend for quartiles
quartile_legend = [mpatches.Patch(color='lightblue', label='25th percentile'),
                   mpatches.Patch(color='royalblue', label='50th percentile (median)'),
                   mpatches.Patch(color='lightblue', label='75th percentile')]
# Remove the legend after plotting
for ax in axes:
    handles, labels = ax.get_legend_handles_labels()  # Get legend handles and labels
    ax.legend_.remove()  # Remove the legend
# Display the legend
#fig.legend(handles=quartile_legend, loc='upper center', bbox_to_anchor=(0.5, 1.08), ncol=3)
fig.legend(handles, ["All ASVs", "Origin", "Atlantic", "Arctic"], loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=4)
#plt.suptitle('Violin Plots for Different Columns', y=1.05, fontsize=16)
# Display the plots
plt.tight_layout()
fig.savefig('figures/Sup_Figure_S15_Arctic_Atlantic_Origin_violin_plot_seperate_ALL.png', bbox_inches='tight')
plt.show()

In [ ]:
df_l5[(df_l5["cluster"]=="03LW") &(df_l5["env"]=="arctic")]

In [ ]:
from scipy import stats
import numpy as np
#results: Clearly, ASVs stable in their natural conditions (and in Atlantic conditions) display a significantly () higher centrality than average, while those predicted to be stable under Arctic conditions show a reduced centrality. 


vector_A =df_l5[(df_l5["cluster"]=="10HS") &(df_l5["env"]=="arctic")]['edge weight nmi (out)'].dropna().values
vector_B =df_l5[(df_l5["cluster"]=="10HS") &(df_l5["env"]=='all ASVs')]['edge weight nmi (out)'].dropna().values

print(" NMI : Arctic (A) < All (B)")
print("A Arctic",np.mean(vector_A))
print("B All",np.mean(vector_B))

t_statistic, p_value = stats.ttest_ind(vector_A, vector_B, equal_var=False, alternative='less')
    
# Define significance level
alpha = 0.05

# Check if the p-value is less than alpha (indicating significance)
if p_value < alpha:
    print("p_value:",round(p_value,3))
    print("Reject the null hypothesis.")
    print("The mean of vector A is significantly greater than the mean of vector B.")
else:
    print("Fail to reject the null hypothesis.")
    print("p_value:",round(p_value,3))
    print("There is not enough evidence to conclude that the mean of vector A is significantly greater than the mean of vector B.")

In [ ]:
from scipy import stats
import numpy as np

#results: Clearly, ASVs stable in their natural conditions (and in Atlantic conditions) display a significantly () higher centrality than average, while those predicted to be stable under Arctic conditions show a reduced centrality. 

vector_B = df_l5[(df_l5["cluster"] == "10HS") & (df_l5["env"] == 'all ASVs')]['edge weight nmi (out)'].dropna().values
vector_A= df_l5[(df_l5["cluster"] == "10HS") & (df_l5["env"] == "atlantic")]['edge weight nmi (out)'].dropna().values
print("NMI: Atlantic vs AlL")
print("All",np.mean(vector_B))
print("Atlantic",np.mean(vector_A))
t_statistic, p_value = stats.ttest_ind(vector_A, vector_B, equal_var=False, alternative='greater')

# Define significance level
alpha = 0.05

# Check if the p-value is less than alpha (indicating significance)
if p_value < alpha:
    print("p_value:", round(p_value, 10))
    print("Reject the null hypothesis.")
    print("The mean of vector A is significantly greater than the mean of vector B.")
else:
    print("Fail to reject the null hypothesis.")
    print("p_value:", round(p_value, 10))
    print(
        "There is not enough evidence to conclude that the mean of vector A is significantly greater than the mean of vector B.")

In [ ]:
from scipy import stats
import numpy as np
#results: Clearly, ASVs stable in their natural conditions (and in Atlantic conditions) display a significantly () higher centrality than average, while those predicted to be stable under Arctic conditions show a reduced centrality. 


vector_A =df_l5[(df_l5["cluster"]=="10HS") &(df_l5["env"]=="arctic")]['Closeness Centrality'].dropna().values
vector_B =df_l5[(df_l5["cluster"]=="10HS") &(df_l5["env"]=='all ASVs')]['Closeness Centrality'].dropna().values

print(" Closeness CentralityArctic (A) > All (B)")
print("A arctic",np.mean(vector_A))
print("B all ",np.mean(vector_B))
t_statistic, p_value = stats.ttest_ind(vector_A, vector_B, equal_var=False, alternative='two-sided')
    
# Define significance level
alpha = 0.05

# Check if the p-value is less than alpha (indicating significance)
if p_value < alpha:
    print("p_value:",round(p_value,3))
    print("Reject the null hypothesis.")
    print("The mean of vector A is significantly greater than the mean of vector B.")
else:
    print("Fail to reject the null hypothesis.")
    print("p_value:",round(p_value,3))
    print("There is not enough evidence to conclude that the mean of vector A is significantly greater than the mean of vector B.")

In [ ]:
from scipy import stats
import numpy as np

#results: Clearly, ASVs stable in their natural conditions (and in Atlantic conditions) display a significantly () higher centrality than average, while those predicted to be stable under Arctic conditions show a reduced centrality. 


vector_A= df_l5[(df_l5["cluster"] == "10HS") & (df_l5["env"] == "atlantic")]['Closeness Centrality'].dropna().values
vector_B = df_l5[(df_l5["cluster"] == "10HS") & (df_l5["env"] == 'all ASVs')]['Closeness Centrality'].dropna().values
print(" Closeness Centrality Atlantic vs all")
print("A atlantic",np.mean(vector_A))
print("B all",np.mean(vector_B))
t_statistic, p_value = stats.ttest_ind(vector_A, vector_B, equal_var=False, alternative='greater')

# Define significance level
alpha = 0.05

# Check if the p-value is less than alpha (indicating significance)
if p_value < alpha:
    print("p_value:", round(p_value, 10))
    print("Reject the null hypothesis.")
    print("The mean of vector A is significantly greater than the mean of vector B.")
else:
    print("Fail to reject the null hypothesis.")
    print("p_value:", round(p_value, 10))
    print(
        "There is not enough evidence to conclude that the mean of vector A is significantly greater than the mean of vector B.")

In [ ]:
## Venn plots

In [ ]:
df_taxa = pd.read_csv("data/F4_euk_taxa_info_table.csv", sep=";")
species_dict = df_taxa[["Unnamed: 0", "Species"]].set_index("Unnamed: 0").to_dict()["Species"]
species_dict


In [ ]:
atlantic_color = '#B82092'
arctic_color = '#2E7FD0'
origin_color = '#5D5FBE'

In [ ]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn3

def plot_venn3(list1, list2, list3, species_dict):
    plt.figure(figsize=(24, 24))
    venn = venn3([set(list1), set(list2), set(list3)], ('Arctic', 'Atlantic = Original', 'Atlantic = Original'))
    venn.get_patch_by_id('100').set_color(arctic_color)
    #venn.get_patch_by_id('010').set_color(atlantic_color)
    #venn.get_patch_by_id('001').set_color(origin_color)
    #venn.get_patch_by_id('101').set_color('#7AAEF3')
    venn.get_patch_by_id('011').set_color('#C58EE9')
    #venn.get_patch_by_id('110').set_color('#D370E2')
    venn.get_patch_by_id('111').set_color('#D9B9F9')
    venn.get_label_by_id('100').set_text('\n'.join([species_dict[species] for species in set(list1) - (set(list2) | set(list3))]))
    #venn.get_label_by_id('010').set_text('\n'.join([species_dict[species] for species in set(list2) - (set(list1) | set(list3))]))
    #venn.get_label_by_id('001').set_text('\n'.join([species_dict[species] for species in set(list3) - (set(list1) | set(list2))]))
    #venn.get_label_by_id('110').set_text('\n'.join([species_dict[species] for species in set(list1) & set(list2) - set(list3)]))
    #venn.get_label_by_id('101').set_text('\n'.join([species_dict[species] for species in set(list1) & set(list3) - set(list2)]))
    venn.get_label_by_id('011').set_text('\n'.join([species_dict[species] for species in set(list2) & set(list3) - set(list1)]))
    venn.get_label_by_id('111').set_text('\n'.join([species_dict[species] for species in set(list1) & set(list2) & set(list3)]))
    #plt.title("Venn Diagram of Species")
    plt.savefig("figures/Sup_Figure_S14_A_ONLYSTABLE_03LW_venn_diagram_proj_vs_non_proj_stable_asvs_species.png", dpi=300)  # Save as PNG with higher resolution
    plt.show()

# Example lists of species
list1 = list_of_arctic_3
list2 = list_of_atlantic_3
list3 = list_of_atlantic_3_NON

plot_venn3(list1, list2, list3, species_dict)

In [ ]:
l3_new_list = list_of_arctic_3 + list_of_atlantic_3 +list_of_atlantic_3_NON
print(len(set(l3_new_list)))
print("list_of_arctic_3",len(set(list_of_arctic_3)))
print("list_of_atlantic_3",len(set(list_of_atlantic_3)))
print("list_of_atlantic_3_NON",len(set(list_of_atlantic_3_NON)))
print("overlap", len(set(list_of_arctic_3).intersection(set(list_of_atlantic_3)).intersection(set(list_of_atlantic_3_NON))))

In [ ]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn3


def plot_venn3(list1, list2, list3, species_dict):
    plt.figure(figsize=(24, 24))
    venn = venn3([set(list1), set(list2), set(list3)], ('Arctic', 'Atlantic = Original', 'Atlantic = Original'))
    #venn.set_colors(['B82092', '#2E7FD0', '#5D5FBE'])
    venn.get_patch_by_id('100').set_color(arctic_color)
    venn.get_patch_by_id('010')  #.set_color('#2E7FD0')
    venn.get_patch_by_id('001')  #.set_color('#5D5FBE')
    venn.get_patch_by_id('111').set_color('#D9B9F9')
    venn.get_patch_by_id('011').set_color('#C58EE9')
    venn.get_label_by_id('100').set_text(
        '\n'.join([species_dict[species] for species in set(list1) - (set(list2) | set(list3))]))
    venn.get_label_by_id(
        '010')  #.set_text('\n'.join([species_dict[species] for species in set(list2) - (set(list1) | set(list3))]))
    venn.get_label_by_id(
        '001')  #.set_text('\n'.join([species_dict[species] for species in set(list3) - (set(list1) | set(list2))]))
    venn.get_label_by_id(
        '110')  #.set_text('\n'.join([species_dict[species] for species in set(list1) & set(list2) - set(list3)]))
    venn.get_label_by_id(
        '101')  #.set_text('\n'.join([species_dict[species] for species in set(list1) & set(list3) - set(list2)]))
    venn.get_label_by_id('011').set_text(
        '\n'.join([species_dict[species] for species in set(list2) & set(list3) - set(list1)]))
    venn.get_label_by_id('111').set_text(
        '\n'.join([species_dict[species] for species in set(list1) & set(list2) & set(list3)]))
    #plt.title("Venn Diagram of Species")
    plt.savefig("figures/Sup_Figure_S14_B_ONLYSTABLE10HS_venn_diagram_proj_vs_non_proj_stable_asvs_species.png",
                dpi=300)  # Save as PNG with higher resolution
    plt.show()


# Example lists of species
list1 = list_of_arctic_10
list2 = list_of_atlantic_10
list3 = list_of_atlantic_10_NON

plot_venn3(list1, list2, list3, species_dict)

In [ ]:
l10_new_list = list_of_arctic_10 + list_of_atlantic_10 +list_of_atlantic_10_NON
print(len(set(l10_new_list)))
print("list_of_arctic_10",len(set(list_of_arctic_10)))
print("list_of_atlantic_10",len(set(list_of_atlantic_10)))
print("list_of_atlantic_10_NON",len(set(list_of_atlantic_10_NON)))
print("overlap", len(set(list_of_arctic_10).intersection(set(list_of_atlantic_10)).intersection(set(list_of_atlantic_10_NON))))